In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import svd, eig
from scipy.ndimage import maximum_filter, minimum_filter
from scipy.spatial import Voronoi, voronoi_plot_2d
from sklearn.cluster import KMeans

# Define grid dimensions
bath = np.loadtxt('grid_1_crop3.dat')
nx, ny = 1322, 1596 
time_steps = 12*4           
Z_time = np.zeros((time_steps, 1596, 1322))
for t in range(time_steps):
    Z_time[t] = np.loadtxt('z'+str(1000+t+1))
    
# Generate time-evolving 
x = np.linspace(0, nx-1, nx)  
y = np.linspace(0, ny-1, ny)  
X, Y = np.meshgrid(x, y)


# Flatten the data across space for DMD analysis
Z_matrix = Z_time.reshape(time_steps, -1).T  # Transpose to get (space x time) structure

# Remove the mean from each row
Z_matrix_mean = np.mean(Z_matrix, axis=1, keepdims=True)
Z_matrix_demeaned = Z_matrix - Z_matrix_mean
print(np.min(Z_matrix_demeaned),np.max(Z_matrix_demeaned))

# Perform Dynamic Mode Decomposition (DMD)
X1 = Z_matrix_demeaned[:, :-1]  # Data matrix at time t
X2 = Z_matrix_demeaned[:, 1:]   # Data matrix at time t+1

# Compute the linear mapping between X1 and X2 using SVD
U, S, Vh = svd(X1, full_matrices=False)

# Compute the approximate linear mapping 
A_tilde = U.T @ X2 @ Vh.T @ np.diag(1 / S)

# Compute eigenvalues and eigenvectors of A_tilde
Lambda, W = eig(A_tilde)
Phi = X2 @ Vh.T @ np.diag(1 / S) @ W  
print("Singular Values (S):", S[:10])  
print("Sum of Singular Values:", np.sum(S))
explained_energy = (S / np.sum(S)) * 100
for i, energy in enumerate(explained_energy[:10]):  # First 10 modes
    print(f"DMD Mode {i+1}: {energy:.2f}% variance explained")

# Reshape the first dominant DMD mode to a spatial field
DMD1_spatial = np.real(Phi[:, 0].reshape(ny, nx))
DMD2_spatial = np.real(Phi[:, 2].reshape(ny, nx))

DMD1 = DMD1_spatial.copy()
DMD2 = DMD2_spatial.copy()

id = np.nonzero(bath<100)
DMD1[id] = np.nan
DMD2[id] = np.nan


FileNotFoundError: grid_1_crop3.dat not found.

In [ ]:
# Identify the multiple local maxima in the first DMD mode
def find_local_extrema(data, neighborhood_size=10):
    local_max = (data == maximum_filter(data, size=neighborhood_size, mode='constant', cval=np.min(data)))
    local_min = (data == minimum_filter(data, size=neighborhood_size))
    return np.where(local_max), np.where(local_min)

maxima,minima = find_local_extrema(DMD1)
maxima2,minima2 = find_local_extrema(DMD2)

# Filter maxima and minima 
def filter_extrema(extrema, i_min=100, i_max=1400, j_min=100, j_max=1200,
                   exclude_i_range=(1, 400), exclude_j_range=(900, 1200)):
    i, j = extrema
    mask = (i >= i_min) & (i <= i_max) & (j >= j_min) & (j <= j_max)
    i = i[mask]
    j = j[mask]
    exclude_mask = ~((i >= exclude_i_range[0]) & (i <= exclude_i_range[1]) &
                     (j >= exclude_j_range[0]) & (j <= exclude_j_range[1]))
    i = i[exclude_mask]
    j = j[exclude_mask]
    return i, j

maxima = filter_extrema(maxima)
minima = filter_extrema(minima)
maxima2 = filter_extrema(maxima2)
minima2 = filter_extrema(minima2)

# Convert real-world coordinates to grid indices
def coordinate_to_index(coord, axis_values):
    return np.argmin(np.abs(axis_values - coord))

# Correct indexing for station selection
max_coords = [(coordinate_to_index(x[j], x), coordinate_to_index(y[i], y)) for i, j in zip(*maxima)]
min_coords = [(coordinate_to_index(x[j], x), coordinate_to_index(y[i], y)) for i, j in zip(*minima)]
max_coords2 = [(coordinate_to_index(x[j], x), coordinate_to_index(y[i], y)) for i, j in zip(*maxima2)]
min_coords2 = [(coordinate_to_index(x[j], x), coordinate_to_index(y[i], y)) for i, j in zip(*minima2)]

# K-Means Clustering 
combined_coords1 = max_coords + min_coords
combined_coords2 = max_coords2 + min_coords2
num_clusters = 30  # Define the number of desired clusters
kmeans = KMeans(n_clusters=num_clusters, random_state=42, n_init=10)
kmeans.fit(combined_coords1)
optimized_stations_kmeans1 = kmeans.cluster_centers_.astype(int)
kmeans.fit(combined_coords2)
optimized_stations_kmeans2 = kmeans.cluster_centers_.astype(int)
